In [248]:
import re
import smtplib
from datetime import datetime
import requests
from bs4 import BeautifulSoup as bs
import os
import re
from random import randint
from time import sleep
from tqdm import tqdm
import mysql.connector

In [228]:
# Get unique values
def unique(xlist):
    unique_list = []
    
    for x in xlist:
        if x not in unique_list:
            unique_list.append(x)
    
    return unique_list

In [229]:
def search(list, keyword):
    for i in range(len(list)):
        if list[i] == keyword:
            return True
    return False

In [230]:
query_url = "https://www.xe.gr/property/results?transaction_name=rent&item_type=re_residence&maximum_price=960&minimum_size=57&geo_lat_from=38.0999672763109&geo_lng_from=24.061083822481493&geo_lat_to=37.792875685371634&geo_lng_to=23.530055641375128&sorting=update_desc"
crafted_urls = []
uniq_urls = []
start_page = 0  # Should be 1 at minimum
max_page_num = 1  # upper limit for the web page number to scrapp
links = []

for i in range(start_page, max_page_num):
    crafted_urls.append( query_url + "&page=" + str(i+1) )
    soup = bs(requests.get(crafted_urls[i]).content, 'lxml')
    
    for a_tag in soup.select('a[href*="enoikiaseis-katoikion"]'):
        links.append(a_tag['href'])
        
    links = links[:-15]


In [231]:
len(links)

68

In [232]:
uniq_urls = unique(links)

In [233]:
len(uniq_urls)

34

In [234]:
blacklist_areas = [
    "nosokomeio-paidwn", 
    "athhna-plateia-amerikhs",
    "vyrwnas",
    "zwgrafoy-kentro",
    "nosokomeio-paidwn",
    "athhna-koyntoyriwtika",
    "athhna-kynosargoys",
    "galatsi",
    "porto-rafth",
    "pagkrati",
    "ippokratoys",
    "ilion",
    "liosia",
    "keratsini"
    ]

In [235]:
blacklist_areas = unique(blacklist_areas)
blacklist_areas

['nosokomeio-paidwn',
 'athhna-plateia-amerikhs',
 'vyrwnas',
 'zwgrafoy-kentro',
 'athhna-koyntoyriwtika',
 'athhna-kynosargoys',
 'galatsi',
 'porto-rafth',
 'pagkrati',
 'ippokratoys',
 'ilion',
 'liosia',
 'keratsini']

In [236]:
def filter_blacklisted_areas(urls, blacklist_areas):
    # Filters based on blacklisted areas
        
    for url in urls:
        for area in blacklist_areas:
            if area in url:
                urls.remove(url)
                break # if blacklisted break the loop
    return urls

In [237]:
cleaned_urls= filter_blacklisted_areas(uniq_urls, blacklist_areas)

In [238]:
print(len(cleaned_urls))

32


In [239]:
class Home:
    def __init__(self, property_url):
        self.property_url = property_url
        self.property_id = -1
        self.bedroom = -1
        self.bathroom = -1
        self.floor = -1
        self.house_reconstruction = 0
        self.building_space = "NA"
        self.compass = "NA"
        self.title = "NA"
        self.price = -1
        self.door = False
        self.canopy = False
        self.heating = "NA"
        self.is_estate = True
        self.description = "NA"
        self.has_oil = False
        self.rejected = True
        
    def set_property_id(self):
        self.property_id = self.property_url.split('/')[6]
        
    def house_print_info(self):
        print(f'Ηλεκτρονική Διεύθυνση: {self.property_url}')
        print(f'Αναγνωριστικό: {self.property_id}')
        print(f'Υπνοδωμάτια: {self.bedroom}')
        print(f'Μπάνια: {self.bathroom}')
        print(f'Όροφος: {self.floor}')
        print(f'Έτος ανακαίνισης: {self.house_reconstruction}')
        print(f'Κατάσταση: {self.building_space}')
        print(f'Προσανατολισμός: {self.compass}')
        print(f'Τίτλος: {self.title}')
        print(f'Τιμή: {self.price}')
        print(f'Πόρτα ασφαλείας: {self.door}')
        print(f'Τέντες: {self.canopy}')
        print(f'Θέρμανση: {self.heating}')
        print(f'Μεσίτης: {self.is_estate}')
        print(f'Περιγραφή: {self.description}')
        print(f'Πετρέλαιο: {self.has_oil}')


    def set_house_info(self, soup_html, soup_lxml):
        
        if search(self.text, "Υπνοδωμάτια:"):
            self.bedroom = int( self.text[self.text.index("Υπνοδωμάτια:") + 2].text )

        if search(self.text, "Μπάνια:"):
            self.bathroom = int( self.text[self.text.index("Μπάνια:") + 2].text )

        floor_data = self.text[self.text.index("Όροφος:") + 2].text
        if type(floor_data) == list and len(floor_data) > 1:
            floor_data = floor_data[0]
        if (floor_data == 'Ισόγειο') or (floor_data == 'Υπερυψωμένο, Ισόγειο') or (floor_data == 'Υπερυψωμένο'):
            self.floor = 0
        elif floor_data == 'Ημιώροφος':
            self.floor = 1 / 2
        elif floor_data == 'Ημιυπόγειο':
            self.floor = -1 / 2
        elif 'Ισόγειο, Ημιυπόγειο':
            self.floor = -1 / 2
        else:
            digits = re.findall(r'\d+', floor_data)
            if type(digits) == list and len(digits) > 0:
                digits = digits[0]
            self.floor = int(str(digits))

        if search(self.text, "Έτος ανακαίνισης:"):
            self.house_reconstruction = int(self.text[self.text.index("Έτος ανακαίνισης:") + 2].text)

        if search(self.text, "Κατάσταση:"):
            self.building_space = self.text[self.text.index("Κατάσταση:") + 2].text

        if search(self.text, "Προσανατολισμός:"):
            self.compass = self.text[self.text.index("Προσανατολισμός:") + 2].text

        self.title = soup_lxml.title.string
        self.price = int (re.findall(r'\d+', soup_lxml.find(class_='price').h2.text)[0])
        self.door = search(self.text, "Πόρτα ασφαλείας")
        self.canopy = search(self.text, "Τέντες")

        if search(self.text, "Θέρμανση:"):
            self.heating = self.text[self.text.index("Θέρμανση:") + 2].text
        
        if self.title.find('Ιδιώτης') != -1:
            self.is_estate = False
        elif len(soup_html.find_all("h2", {"class": "title"})) == 0:
            self.is_estate = True
        

        if self.description:
            if ((self.description.find('πετρέλαιο') != -1) or (self.description.find('πετρελαίου') != -1)):
                self.has_oil = True
        if search(self.text, "Μέσο θέρμανσης:"):
            self.heating = self.text[self.text.index("Μέσο θέρμανσης:") + 2].text
            if self.heating == "Πετρέλαιο" or self.heating == "πετρελαίου":
                self.has_oil = True
            
    def set_text(self, soup_html):
        parent = soup_html.find("body").find("ul")  # finding parent <ul> tag
        self.text = list(parent.descendants)  # finding all <li> tags
        
    def validate(self):
        if not self.has_oil and not self.is_estate:
            self.rejected = False

In [240]:
def make_homes(urls):
    Homes = []

    for i in range(0, len(urls)):
        Homes.append(Home(urls[i]))
        Homes[i].set_property_id()
    
    return Homes

In [241]:
Homes = make_homes(cleaned_urls)
len(Homes)

32

In [242]:
print(Homes[3].property_id)
print(Homes[3].property_url)

786199120
https://www.xe.gr/property/d/enoikiaseis-katoikion/786199120/karellas-580-95?first_in_widget_ad_id=786199247&rank_in_widget=4&widget_name=search+results+list


In [243]:
def get_soup_objects(url):
    
    # creating requests object
    html = requests.get(url).content
        
    # creating soup objects
    soup_html = bs(html, 'html.parser')
    soup_lxml = bs(html, 'lxml')
    
    return [soup_html, soup_lxml]

In [244]:
def parse_and_filter_homes():
    selected_houses = []
    selected = False
    
    for h in Homes:
        delay = randint(1,5)
        print(f'\nProcessing house with id: {h.property_id} {h.property_url} (sleeping for {delay} seconds)')
        sleep(delay)
        
        [soup_html, soup_lxml] = get_soup_objects(h.property_url)
        
        # find tags and set text
        h.set_text(soup_html) 
        
        h.set_house_info(soup_html, soup_lxml)
        
        h.validate()
        
        if not h.rejected:
            h.house_print_info()
            selected_houses.append(h)
            print(f'This house matches out criteria!')
            print(f'Addind it to our database...')
            print(f'Informing you via email...')
        else:
            print(f'{h.property_id} has been rejected')
        
    
    return selected_houses

In [245]:
selected_houses = parse_and_filter_homes()


Processing house with id: 786199247 https://www.xe.gr/property/d/enoikiaseis-katoikion/786199247/athhna-ghrokomeio-670-67?first_in_widget_ad_id=786199247&rank_in_widget=1&widget_name=search+results+list (sleeping for 2 seconds)
786199247 has been rejected

Processing house with id: 786199223 https://www.xe.gr/property/d/enoikiaseis-katoikion/786199223/zwgrafoy-ilisia-850-110?first_in_widget_ad_id=786199247&rank_in_widget=2&widget_name=search+results+list (sleeping for 3 seconds)
786199223 has been rejected

Processing house with id: 786199203 https://www.xe.gr/property/d/enoikiaseis-katoikion/786199203/nikaia-neapolh-650-95?first_in_widget_ad_id=786199247&rank_in_widget=3&widget_name=search+results+list (sleeping for 4 seconds)
786199203 has been rejected

Processing house with id: 786199120 https://www.xe.gr/property/d/enoikiaseis-katoikion/786199120/karellas-580-95?first_in_widget_ad_id=786199247&rank_in_widget=4&widget_name=search+results+list (sleeping for 2 seconds)
786199120 has

786196409 has been rejected

Processing house with id: 786196405 https://www.xe.gr/property/d/enoikiaseis-katoikion/786196405/peiraias-freattyda-750-93?first_in_widget_ad_id=786199247&rank_in_widget=24&widget_name=search+results+list (sleeping for 3 seconds)
786196405 has been rejected

Processing house with id: 786196392 https://www.xe.gr/property/d/enoikiaseis-katoikion/786196392/nikaia-neapolh-660-85?first_in_widget_ad_id=786199247&rank_in_widget=25&widget_name=search+results+list (sleeping for 2 seconds)
786196392 has been rejected

Processing house with id: 786196368 https://www.xe.gr/property/d/enoikiaseis-katoikion/786196368/korydallos-nekrotafeio-nikaias-799-93?first_in_widget_ad_id=786199247&rank_in_widget=26&widget_name=search+results+list (sleeping for 2 seconds)
786196368 has been rejected

Processing house with id: 786196254 https://www.xe.gr/property/d/enoikiaseis-katoikion/786196254/korydallos-nekrotafeio-nikaias-699-75?first_in_widget_ad_id=786199247&rank_in_widget=27&w

In [246]:
h = selected_houses[1]

In [249]:
def insert_variables_into_table(property_id, property_url, bedroom, bathroom, floor, house_reconstruction,
                                building_space, compass, title, price, door, canopy, heating, description, has_oil):
    connection = None
    try:
        # In MySQL 8.0, caching_sha2_password is the default authentication plugin rather than mysql_native_password.
        connection = mysql.connector.connect(host='localhost',
                                             database='xedb',
                                             user='root',
                                             password='dxcfvgbhnjm4567ctfvghj&*(klYFlCnmPzyfvgbh',
                                             auth_plugin='mysql_native_password')
        cursor = connection.cursor()
        mySql_insert_query = """INSERT INTO homes (property_id, property_url, bedroom, bathroom, floor, 
                                    house_reconstruction, building_space, compass, title, price, door, canopy, heating,
                                    description, has_oil) 
                                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) """

        record = (property_id, property_url, bedroom, bathroom, floor, house_reconstruction, building_space, compass,
                  title, price, door, canopy, heating, description, has_oil)
        cursor.execute(mySql_insert_query, record)
        connection.commit()
        print("Record inserted successfully into homes table")

    except mysql.connector.Error as error:
        print("Failed to insert into MySQL table {}".format(error))

    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")



In [268]:
insert_variables_into_table(h.property_id, h.property_url, h.bedroom, h.bathroom, h.floor, h.house_reconstruction,
                                h.building_space, h.compass, h.title, h.price, h.door, h.canopy, h.heating, h.description, h.has_oil)

Record inserted successfully into homes table
MySQL connection is closed


In [223]:
h.property_url

'https://www.xe.gr/property/d/enoikiaseis-katoikion/784159921/nea-iwnia-perissos-550-57?first_in_widget_ad_id=786198290&rank_in_widget=2&widget_name=search+results+list'

In [224]:
query="SELECT * FROM `homes` WHERE property_id = '786192763'"

In [260]:
def db_house_exists(property_id):
    connection = None
    try:
        # In MySQL 8.0, caching_sha2_password is the default authentication plugin rather than mysql_native_password.
        connection = mysql.connector.connect(host='localhost',
                                             database='xedb',
                                             user='root',
                                             password='dxcfvgbhnjm4567ctfvghj&*(klYFlCnmPzyfvgbh',
                                             auth_plugin='mysql_native_password')
        cursor = connection.cursor()
        query="SELECT * FROM `homes` WHERE property_id = 785465122"

        cursor.execute(query) #, (property_id))
        
        print(cursor)

        for (property_id, property_url, bedroom, bathroom, floor, house_reconstruction,
             building_space, compass, title, price, door, canopy, 
             heating, description, has_oil) in cursor:
            print(f'{property_id} exists in db home table homes')
    except mysql.connector.Error as error:
        print("Failed to query MySQL table {}".format(error))
        
    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

In [277]:
connection = mysql.connector.connect(host='localhost',
                                     database='xedb',
                                     user='root',
                                     password='dxcfvgbhnjm4567ctfvghj&*(klYFlCnmPzyfvgbh',
                                     auth_plugin='mysql_native_password')
cursor = connection.cursor()

query=f'SELECT property_id FROM `homes` WHERE property_id = {h.property_id}'

res = cursor.execute(query)
    
#print(cursor)

#for (property_id, property_url, bedroom, bathroom, floor, house_reconstruction,
#     building_space, compass, title, price, door, canopy, 
#     heating, description, has_oil) in cursor:
    
#    print(f'{property_id} exists in db home table homes')

cursor.fetchone()
#for property_id in cursor:
#    print(f'{property_id} exists in db')

cursor.close()
connection.close()

In [269]:
for hs in selected_houses:
    insert_variables_into_table(h.property_id, h.property_url, h.bedroom, h.bathroom, h.floor, 
                                h.house_reconstruction, h.building_space, h.compass, h.title, 
                                h.price, h.door, h.canopy, h.heating, h.description, h.has_oil
                               )

Failed to insert into MySQL table 1062 (23000): Duplicate entry '786198290' for key 'PRIMARY'
MySQL connection is closed
Failed to insert into MySQL table 1062 (23000): Duplicate entry '786198290' for key 'PRIMARY'
MySQL connection is closed
Failed to insert into MySQL table 1062 (23000): Duplicate entry '786198290' for key 'PRIMARY'
MySQL connection is closed
Failed to insert into MySQL table 1062 (23000): Duplicate entry '786198290' for key 'PRIMARY'
MySQL connection is closed
Failed to insert into MySQL table 1062 (23000): Duplicate entry '786198290' for key 'PRIMARY'
MySQL connection is closed


In [251]:
db_house_exists(h.property_id)

MySQL connection is closed


ValueError: Could not process parameters